<a href="https://colab.research.google.com/github/git-hatano/pytorch_advanced/blob/main/1_image_classification/1_5_fine_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1-5 ファインチューニングの実装

In [ ]:
import os
import urllib.request
import zipfile

## データの用意

In [ ]:
# フォルダ「data」が存在しない場合は作成する
data_dir = "./data/"
if not os.path.exists(data_dir):
    os.mkdir(data_dir)

In [ ]:
# 1.3節で使用するアリとハチの画像データをダウンロードし解凍します
# PyTorchのチュートリアルで用意されているものです
# https://pytorch.org/tutorials/beginner/transfer_learning_tutorial.html

url = "https://download.pytorch.org/tutorial/hymenoptera_data.zip"
save_path = os.path.join(data_dir, "hymenoptera_data.zip")

if not os.path.exists(save_path):
    urllib.request.urlretrieve(url, save_path)

    # ZIPファイルを読み込み
    zip = zipfile.ZipFile(save_path)
    zip.extractall(data_dir)  # ZIPを解凍
    zip.close()  # ZIPファイルをクローズ

    # ZIPファイルを消去
    os.remove(save_path)

## 前処理

In [ ]:
# パッケージのimport
import glob
import os.path as osp
import random
import numpy as np
import json
from PIL import Image
from tqdm import tqdm
import matplotlib.pyplot as plt
%matplotlib inline

import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import torchvision
from torchvision import models, transforms

# 乱数のシードを設定
torch.manual_seed(1234)
np.random.seed(1234)
random.seed(1234)

### ImageTransform

In [ ]:
#入力画像の前処理をするクラス
#訓練時と推論時で処理が異なる

class ImageTransform():
    """
    画像の前処理クラス。訓練時、検証時で異なる動作をする。
    画像のサイズをリサイズし、色を標準化する。
    訓練時はRandomResizeCropとRandamHorizonalFlipでデータオーギュメンテーションする。

    Attributes
    -------------------
    resize: int
        リサイズ先の画像の大きさ
    mean: (R, G, B)
        各色チャンネルの平均値
    std: (R, G, B)
        各色チャンネルの標準偏差
    """

    def __init__(self, resize, mean, std):
        self.data_transform = {
            'train': transforms.Compose([
                transforms.RandomResizedCrop(resize, scale=(0.5, 1.0)),
                transforms.RandomHorizontalFlip(),
                transforms.ToTensor(),
                transforms.Normalize(mean, std)
            ]),
            'val': transforms.Compose([
                transforms.Resize(resize),
                transforms.CenterCrop(resize),
                transforms.ToTensor(),
                transforms.Normalize(mean, std)
            ])
        }
    
    def __call__(self, img, phase='train'):
        """
        Parameters
        ----------------
        phase: 'train' or 'val'
            前処理のモードを設定
        """
        return self.data_transform[phase](img)

### make_datapath_list

In [ ]:
#画像へのファイルパスのリストを作成する
def make_datapath_list(phase='train'):
    """
    データのパスを格納したリストを作成する。
    Parameters
    --------------------------
    phase: 'train' or 'val'
        訓練データか検証データかを指定する
    Returns
    --------------------------
    path_list: list
        データへのパスを格納したリスト
    """
    rootpath = "/content/data/hymenoptera_data"
    target_path = osp.join(rootpath+"/"+phase+"/**/*.jpg")
    # print(target_path)

    path_list = []
    for path in glob.glob(target_path):
        path_list.append(path)
    return path_list

### HymenopteraDataset

In [ ]:
#Datasetを作成
class HymenopteraDataset(data.Dataset):
    """
    Datasetのクラス。PyTorchのデータセットクラスを継承。
    Attributes
    --------------------------
    file_list: list
        画像のパスを格納したリスト
    transform: object
        前処理クラスのインスタンス
    phase: 'train' or 'val'
        訓練か検証かを指定
    """
    def __init__(self, file_list, transform=None, phase='train'):
        self.file_list = file_list
        self.transform = transform
        self.phase = phase
    
    def __len__(self):
        """画像の枚数を返す"""
        return len(self.file_list)
    
    def __getitem__(self, index):
        """前処理した画像のTensor形式のデータとラベルを取得"""
        #index版目の画像をロード
        img_path = self.file_list[index]
        img = Image.open(img_path)

        #画像の前処理を実行
        img_transformed = self.transform(img, self.phase)

        # 画像のラベルをファイル名から抜き出す
        if self.phase == "train":
            label = img_path.split("/")[-2]
        elif self.phase == "val":
            label = img_path.split("/")[-2]

        # ラベルを数値に変更する
        if label == "ants":
            label = 0
        elif label == "bees":
            label = 1

        return img_transformed, label

## Dataset, DataLoaderを作成

In [ ]:
#画像へのパスのリストを作成
train_list = make_datapath_list(phase='train')
val_list = make_datapath_list(phase='val')

#Datasetを作成
size = 224
mean = (0.485, 0.456, 0.406)
std = (0.229, 0.224, 0.225)

train_dataset = HymenopteraDataset(file_list=train_list, transform=ImageTransform(size, mean, std), phase='train')
val_dataset = HymenopteraDataset(file_list=val_list, transform=ImageTransform(size, mean, std), phase='val')

#DataLoaderを作成
batch_size = 32
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

dataloaders_dict = {"train": train_dataloader, "val": val_dataloader}

#動作確認
batch_iterator = iter(dataloaders_dict["train"])
inputs, labels = next(batch_iterator)
print(inputs.size())
print(labels)

torch.Size([32, 3, 224, 224])
tensor([1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1,
        1, 1, 0, 0, 0, 0, 1, 1])


## modelを作成

In [ ]:
#学習済VGG16をロード
use_pretrained = True
net = models.vgg16(pretrained=use_pretrained)

#VGG16の最後の出力層の出力ユニットを2クラスに付け替え
net.classifier[6] = nn.Linear(in_features=4096, out_features=2)

#訓練モードに設定
net.train()
print('ネットワーク設定完了：学習済みの重みをロードし、訓練モードに設定しました')

/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth


  0%|          | 0.00/528M [00:00<?, ?B/s]

ネットワーク設定完了：学習済みの重みをロードし、訓練モードに設定しました


##損失関数を定義

In [ ]:
criterion = nn.CrossEntropyLoss()

## 最適化手法を設定

In [ ]:
#学習させるパラメータを格納
#今回は層によって学習率を変えるために分けて格納
params_to_update1 = []
params_to_update2 = []
params_to_update3 = []

#学習させる層のパラメータ名を指定
update_param_names_1 = ["features"]
update_param_names_2 = ["classifier.0.weight", "classifier.0.bias", "classifier.3.weight", "classifier.3.bias"]
update_param_names_3 = ["classifier.6.weight", "classifier.6.bias"]

#パラメータごとに各リストに格納
for name, param in net.named_parameters():
    if update_param_names_1[0] in name:
        param.requires_grad = True
        params_to_update1.append(param)
        print("params_to_update1に格納 :", name)
    
    elif name in update_param_names_2:
        param.requires_grad = True
        params_to_update2.append(param)
        print("params_to_update_2に格納 :", name)
    
    elif name in update_param_names_3:
        param.requires_grad = True
        params_to_update3.append(param)
        print("params_to_update_3に格納 :", name)
    
    else:
        param.requires_grad = False
        print("勾配計算なし。学習しない :", name)

params_to_update1に格納 : features.0.weight
params_to_update1に格納 : features.0.bias
params_to_update1に格納 : features.2.weight
params_to_update1に格納 : features.2.bias
params_to_update1に格納 : features.5.weight
params_to_update1に格納 : features.5.bias
params_to_update1に格納 : features.7.weight
params_to_update1に格納 : features.7.bias
params_to_update1に格納 : features.10.weight
params_to_update1に格納 : features.10.bias
params_to_update1に格納 : features.12.weight
params_to_update1に格納 : features.12.bias
params_to_update1に格納 : features.14.weight
params_to_update1に格納 : features.14.bias
params_to_update1に格納 : features.17.weight
params_to_update1に格納 : features.17.bias
params_to_update1に格納 : features.19.weight
params_to_update1に格納 : features.19.bias
params_to_update1に格納 : features.21.weight
params_to_update1に格納 : features.21.bias
params_to_update1に格納 : features.24.weight
params_to_update1に格納 : features.24.bias
params_to_update1に格納 : features.26.weight
params_to_update1に格納 : features.26.bias
params_to_update1に格納 : f

In [ ]:
#最適化手法の設定
optimizer = optim.SGD([
    {"params": params_to_update1, "lr": 1e-4},
    {"params": params_to_update2, "lr": 5e-4},
    {"params": params_to_update3, "lr": 1e-3},
], momentum=0.9)

## 学習と検証

In [ ]:
#モデルを学習させる関数
def train_model(net, dataloaders_dict, criterion, optimizer, num_epochs):
    #初期設定
    #GPUが使えるかを確認
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print(f"using devide : {device}")
    #ネットワークをGPUへ
    net.to(device)
    #ネットワークがある程度固定ならば高速化させる
    torch.backends.cudnn.benchmark = True

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch+1, num_epochs))
        print('-------------')

        #epochごとに学習と検証をループ
        for phase in ['train', 'val']:
            if phase=='train':
                net.train()
            else:
                net.eval()

            epoch_loss = 0.0
            epoch_corrects = 0
            #未学習時の検証性能を確かめるため、epoch=0の学習は省略
            if epoch==0 and phase=='train':
                continue
            
            #dataloaderからミニバッチを取り出すループ
            for inputs, labels in tqdm(dataloaders_dict[phase]):
                #GPUが使えるならデータを送る
                inputs = inputs.to(device)
                labels = labels.to(device)
                #optimizerを初期化
                optimizer.zero_grad()

                #forward
                with torch.set_grad_enabled(phase=='train'):
                    outputs = net(inputs)
                    loss = criterion(outputs, labels)
                    _, preds = torch.max(outputs, 1)#ラベルを予測

                    #学習時はbackwardで勾配計算、パラメータの更新
                    if phase=='train':
                        loss.backward()
                        optimizer.step()
                    
                    #lossの合計を更新
                    epoch_loss += loss.item()*inputs.size(0)
                    #正解数の合計を更新
                    epoch_corrects += torch.sum(preds==labels.data)
            
            #epochごとのlossと正解率を表示
            epoch_loss = epoch_loss / len(dataloaders_dict[phase].dataset)
            epoch_acc = epoch_corrects.double() / len(dataloaders_dict[phase].dataset)
            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

In [ ]:
num_epochs = 2
train_model(net, dataloaders_dict, criterion, optimizer, num_epochs)

using devide : cuda:0
Epoch 1/2
-------------


100%|██████████| 5/5 [00:10<00:00,  2.01s/it]


val Loss: 0.6151 Acc: 0.6405
Epoch 2/2
-------------


100%|██████████| 8/8 [00:10<00:00,  1.27s/it]


train Loss: 0.4404 Acc: 0.7654


100%|██████████| 5/5 [00:01<00:00,  2.81it/s]

val Loss: 0.1950 Acc: 0.9412


#学習したネットワークを保存・ロード

In [ ]:
#パラメータの保存
save_path = "./weights_fine_tuning.pth"
torch.save(net.state_dict(), save_path)

In [ ]:
#パラメータのロード
load_path = "./weights_fine_tuning.pth"
load_weights = torch.load(load_path)
net.load_state_dict(load_weights)

#GPU上で保存された重みをCPU上でロードする場合
# load_weights = torch.load(load_path, map_location={"cuda:0": "cpu"})
# net.load_state_dict(load_weights)

<All keys matched successfully>